# Simulation experiment using noisy data 

Run entire simulation experiment multiple times to generate confidence interval.  The simulation experiment can be found in ```functions/pipeline.py```

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
dataset_name = "Pseudomonas_analysis"
analysis_name = 'Pa_sample_lvl_sim'
NN_architecture = 'NN_2500_30'
file_prefix = "Experiment"
num_simulated_samples = 6000
lst_num_experiments = [1, 2, 5, 10, 20,
                     50, 100, 500, 1000, 2000, 3000, 6000]

corrected = False
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input file
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,    
    "data",
    "input",
    "train_set_normalized.pcl")

In [4]:
# Output files
similarity_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Pa_sample_lvl_sim_similarity_uncorrected.pickle")

ci_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Pa_sample_lvl_sim_ci_uncorrected.pickle")

similarity_permuted_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Pa_sample_lvl_sim_permuted")

In [5]:
# Run multiple simulations
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.sample_level_simulation_uncorrected)(i,
                                                       NN_architecture,
                                                       dataset_name,
                                                       analysis_name,
                                                       num_simulated_samples,
                                                       lst_num_experiments,
                                                       corrected,
                                                       use_pca,
                                                       num_PCs,
                                                       file_prefix,
                                                       normalized_data_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 57.1min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 57.1min remaining: 85.7min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 57.3min remaining: 38.2min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 57.3min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 57.3min finished


In [6]:
# permuted score
permuted_score = results[0][0]

In [7]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of experiments,,,,,
1,0.999994,0.999994,0.999994,0.999994,0.999994
2,0.901760,0.900300,0.901131,0.902436,0.900573
5,0.607757,0.606824,0.607563,0.609491,0.610785
10,0.127849,0.131282,0.131376,0.131820,0.133415
20,0.049286,0.046376,0.045556,0.044243,0.038659
50,0.151226,0.149932,0.134562,0.148625,0.149264
100,0.241074,0.238559,0.237977,0.243663,0.237749
500,0.719821,0.721868,0.712278,0.719117,0.707928
1000,0.940604,0.934020,0.934988,0.907385,0.933704


In [8]:
# Get mean svcca score for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999994
2,0.901240
5,0.608484
10,0.131148
20,0.044824
50,0.146722
100,0.239804
500,0.716203
1000,0.930140


In [9]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,3.739870e-08
2,2.757877e-04
5,5.109224e-04
10,6.431385e-04
20,1.236970e-03
50,2.171071e-03
100,8.002745e-04
500,1.854375e-03
1000,4.118726e-03


In [10]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval 
err = std_scores*1.96

In [11]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999994,0.999994
2,0.900700,0.901781
5,0.607483,0.609485
10,0.129888,0.132409
20,0.042399,0.047248
50,0.142467,0.150977
100,0.238236,0.241373
500,0.712568,0.719837
1000,0.922068,0.938213


In [12]:
mean_scores

,score
number of experiments,
1,0.999994
2,0.901240
5,0.608484
10,0.131148
20,0.044824
50,0.146722
100,0.239804
500,0.716203
1000,0.930140


In [13]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)
np.save(similarity_permuted_file, permuted_score)